In [3]:
from rdkit import Chem, DataStructs
from rdkit.Chem import AllChem
from rdkit.Chem import Draw
from rdkit.Chem import rdMolDescriptors
from rdkit import RDLogger
from envs.sascorer import calculateScore

In [2]:
import numpy as np
import os
import pandas as pd
from sklearn.model_selection import train_test_split
import json
from tqdm import tqdm
import re
from datetime import datetime

In [4]:
dmnp_smiles = 'CC1CCC(C2=C1C=CC(=C2)C)C(C)CCC(=O)O'
calculateScore(Chem.MolFromSmiles(dmnp_smiles))

[21:27:06] DEPRECATION WARNING: please use MorganGenerator


3.3559737244164882

In [33]:
path = './docking/DS_data/'
ds = np.empty(0, dtype=float)
cnt = 0

for i in range(10):
    loc_path = path + f'{i * 25}k/'
    print('Загрузка из ', loc_path)
    files = sorted(os.listdir(loc_path), key=lambda x: int(re.findall(r'\d+', x)[0]))
    print(files)
    if len(files) < 25:
        print('Alarm: ', loc_path)
    for file in files:
        fl = re.findall(r'\d+', file)
        if int(fl[0]) // 1000 != cnt:
            print('Miss in {} thousand'.format(cnt))
            print(cnt, fl[0])
        a_new = np.load(loc_path + file, allow_pickle=True)
        ds = np.concat([ds, a_new])
        if a_new.shape[0] != 1000:
            print('Alarm: ', file)
        cnt += 1

ds.dump(path + 'docking_scores_all.npy')

Загрузка из  ./docking/DS_data/0k/
['docking_scores_0-999.npy', 'docking_scores_1000-1999.npy', 'docking_scores_2000-2999.npy', 'docking_scores_3000-3999.npy', 'docking_scores_4000-4999.npy', 'docking_scores_5000-5999.npy', 'docking_scores_6000-6999.npy', 'docking_scores_7000-7999.npy', 'docking_scores_8000-8999.npy', 'docking_scores_9000-9999.npy', 'docking_scores_10000-10999.npy', 'docking_scores_11000-11999.npy', 'docking_scores_12000-12999.npy', 'docking_scores_13000-13999.npy', 'docking_scores_14000-14999.npy', 'docking_scores_15000-15999.npy', 'docking_scores_16000-16999.npy', 'docking_scores_17000-17999.npy', 'docking_scores_18000-18999.npy', 'docking_scores_19000-19999.npy', 'docking_scores_20000-20999.npy', 'docking_scores_21000-21999.npy', 'docking_scores_22000-22999.npy', 'docking_scores_23000-23999.npy', 'docking_scores_24000-24999.npy']
Загрузка из  ./docking/DS_data/25k/
['docking_scores_25000-25999.npy', 'docking_scores_26000-26999.npy', 'docking_scores_27000-27999.npy',

Загрузка из  ./docking/DS_data/175k/
['docking_scores_175000-175999.npy', 'docking_scores_176000-176999.npy', 'docking_scores_177000-177999.npy', 'docking_scores_178000-178999.npy', 'docking_scores_179000-179999.npy', 'docking_scores_180000-180999.npy', 'docking_scores_181000-181999.npy', 'docking_scores_182000-182999.npy', 'docking_scores_183000-183999.npy', 'docking_scores_184000-184999.npy', 'docking_scores_185000-185999.npy', 'docking_scores_186000-186999.npy', 'docking_scores_187000-187999.npy', 'docking_scores_188000-188999.npy', 'docking_scores_189000-189999.npy', 'docking_scores_190000-190999.npy', 'docking_scores_191000-191999.npy', 'docking_scores_192000-192999.npy', 'docking_scores_193000-193999.npy', 'docking_scores_194000-194999.npy', 'docking_scores_195000-195999.npy', 'docking_scores_196000-196999.npy', 'docking_scores_197000-197999.npy', 'docking_scores_198000-198999.npy', 'docking_scores_199000-199999.npy']
Загрузка из  ./docking/DS_data/200k/
['docking_scores_200000-2

In [39]:
ds[ds != 0].shape

(246162,)

In [34]:
from envs.sascorer import calculateScore

file_path = os.path.join('./dataset/zinc250k/zinc250k.smi')
fp = open(file_path, 'r')
smiles_list = [smiles.strip() for smiles in fp]

sas = lambda sml: calculateScore(Chem.MolFromSmiles(sml))
sas_vec = np.vectorize(sas)

def tanimoto(sml):
    
    mol_dmnp = Chem.MolFromSmiles('CC1CCC(C2=C1C=CC(=C2)C)C(C)CCC(=O)O')
    fp_dmnp = AllChem.GetMorganFingerprint(mol_dmnp, 2)

    mol_sml = Chem.MolFromSmiles(sml)
    fp_sml = AllChem.GetMorganFingerprint(mol_sml, 2) # здесь возможно возникает то самое предупреждение с MorganFingerprint
    
    return DataStructs.TanimotoSimilarity(fp_dmnp, fp_sml) # здесь вставить свой dist

tanimoto_vec = np.vectorize(tanimoto)

sa = sas_vec(smiles_list)
td = tanimoto_vec(smiles_list)

In [35]:
dset = pd.DataFrame({'SMILES': np.array(smiles_list), 'DS': ds, 'SA': sa, 'TD': td})

In [44]:
dset_clear = dset.iloc[np.where(ds != 0)]

In [56]:
dset_clear.to_csv(path + 'docking_dataset.csv', index=False)

In [74]:
dset_clear['DS'].mean()

np.float64(-7.371396478741642)

In [71]:
dset_clear['DS'][:5]

0   -8.6
1   -8.3
2   -8.9
3   -8.3
4   -7.0
Name: DS, dtype: float64

In [77]:
dset_clear['DS'].max(), dset_clear['DS'].min()

(np.float64(55.9), np.float64(-14.4))

In [80]:
dset_clear['SA'].min(), dset_clear['SA'].max()

(np.float64(1.1327382804544328), np.float64(7.289282840617412))

In [105]:
(-dset_clear['SA']).std(), (dset_clear['SA']).std()

(np.float64(0.830780685305625), np.float64(0.830780685305625))

In [103]:
(( -dset_clear['SA'] + dset_clear['SA'].mean()) / (-dset_clear['SA']).std())

0         1.154338
1        -0.468123
2         0.689068
3         0.265224
4        -1.194161
            ...   
249451    0.171370
249452    0.862405
249453    1.447379
249454    1.225905
249455    0.997673
Name: SA, Length: 246162, dtype: float64

In [98]:
calculateScore(Chem.MolFromSmiles('C1=CC=C2C(=C1)C=CC3=C2C=CC(=C3)C(=O)N4CCN(CC4)C5=CC=CC=C5'))

1.820501705923089

In [83]:
with open('./data_preprocessed/zinc250k/idx.json', 'w') as f:
    tr, tst = train_test_split(list(range(1,len(dset_clear))), test_size=0.1, random_state=42)
    json.dump([tr, tst], f)

In [54]:
dt = datetime.now()
'r2_tr_{}_{:02d}-{:02d}-{:02d}'.format(dt.date(), dt.hour, dt.minute, dt.second)

'r2_tr_2025-04-16_22-14-27'

In [ ]:
type(Chem.MolFromSmiles('CCO'))

In [4]:
moses = pd.read_csv('dataset_v1.csv')

In [5]:
moses_smiles = moses['SMILES'].values.tolist()

In [ ]:
moses_smiles

In [ ]:
np.where(td > 0.3)

In [ ]:
td2.shape

In [ ]:
def tanimoto(sml):

    # fp_gen = rdMolDescriptors.GetMorganGenerator(radius=2, nBits=2048)
    
    mol_dmnp = Chem.MolFromSmiles('CC1CCC(C2=C1C=CC(=C2)C)C(C)CCC(=O)O')
    fp_dmnp = AllChem.GetMorganFingerprint(mol_dmnp, 2)
    # fp_dmnp = fp_gen.GetFingerprint(mol_dmnp, 2)

    mol_sml = Chem.MolFromSmiles(sml)
    fp_sml = AllChem.GetMorganFingerprint(mol_sml, 2) # здесь возможно возникает то самое предупреждение с MorganFingerprint
    # fp_sml = fp_gen.GetFingerprint(mol_sml, 2)
    
    return DataStructs.TanimotoSimilarity(fp_dmnp, fp_sml) # здесь вставить свой dist

tanimoto_vec = np.vectorize(tanimoto)

# td2 = tanimoto_vec(moses_smiles[:500000])

In [19]:
max_td = 0
n = 0

rdLogger = RDLogger.logger()
rdLogger.setLevel(RDLogger.ERROR)

def tanimoto(sml):
    
    mol_dmnp = Chem.MolFromSmiles(smiles_list[100000])
    fp_dmnp = AllChem.GetMorganFingerprint(mol_dmnp, 2)

    mol_sml = Chem.MolFromSmiles(sml)
    fp_sml = AllChem.GetMorganFingerprint(mol_sml, 2) # здесь возможно возникает то самое предупреждение с MorganFingerprint
    
    return DataStructs.TanimotoSimilarity(fp_dmnp, fp_sml)


for i, smile in enumerate(tqdm(smiles_list[100001:])):
    tan_di = tanimoto(smile)
    if tan_di > max_td:
        max_td = tan_di
    if tan_di > 0.5:
        n += 1
    if i % 10000 == 0:
        print('max_td: {}, n: {}'.format(max_td, n))

  0%|          | 0/149455 [00:00<?, ?it/s]

  0%|          | 142/149455 [00:00<03:56, 631.19it/s]

max_td: 0.1487603305785124, n: 0


  7%|▋         | 10170/149455 [00:12<02:49, 819.57it/s]

max_td: 0.39361702127659576, n: 0


...

 87%|████████▋ | 130178/149455 [02:40<00:24, 796.62it/s]

max_td: 0.6444444444444445, n: 1


 94%|█████████▍| 140168/149455 [02:53<00:11, 819.09it/s]

max_td: 0.6444444444444445, n: 2


100%|██████████| 149455/149455 [03:04<00:00, 809.15it/s]


In [ ]:
np.load('docking_scores_0-9.npy', allow_pickle=True)

In [ ]:
ds = np.load('docking_scores.npy', allow_pickle=True)
ds[ds != 0][797:798].min()

У train №797 DS -9.8

In [ ]:
X = np.load('./data_preprocessed/zinc250k/node_features.npy')
print('Nodes done')
A = np.load('./data_preprocessed/zinc250k/adj_features.npy')
print('Bonds done')

In [ ]:
mol_sizes = np.load('./data_preprocessed/zinc1500k/mol_sizes.npy')

In [ ]:
DMNP_mol = Chem.MolFromPDBFile("./docking/294-BD_AD_receptor_ligand_140.988_160.730_-16.038/294-BD_AD_receptor_ligand_140.988_160.730_-16.038.pdb")

In [ ]:
DMNP_smiles = Chem.MolToSmiles(DMNP_mol)
DMNP_smiles

In [ ]:
import time

In [ ]:
import os
current_dir = os.getcwd()  # Текущая рабочая директория
parent_dir = os.path.dirname(current_dir)
file_path = os.path.join(parent_dir, 'title.txt')

In [ ]:
current_dir, parent_dir, file_path

In [ ]:
from rdkit import Chem
from rdkit.Chem import AllChem
import subprocess

# protein = Chem.MolFromPDBFile("./docking/294-BD_AD_receptor_ligand_140.988_160.730_-16.038/receptor.pdb")
# Chem.MolToPDBFile(protein, "receptor_rdkit.pdbqt")

def pdb_to_pdbqt_rdkit(input_pdb, output_pdbqt):
    """
    Конвертирует PDB в PDBQT с помощью RDKit (подготовка) и Open Babel (финальная конвертация).
    
    Параметры:
        input_pdb (str): Путь к входному PDB-файлу.
        output_pdbqt (str): Путь для сохранения PDBQT-файла.
    """
    # 1. Загрузка PDB в RDKit
    print(1)
    mol = Chem.MolFromPDBFile(input_pdb, removeHs=False)
    if mol is None:
        raise ValueError("Не удалось загрузить PDB-файл. Проверьте его целостность.")

    # 2. Добавление водородов (если их нет)
    print(2)
    mol = Chem.AddHs(mol, addCoords=True)

    # 3. Оптимизация геометрии (MMFF94 или UFF)
    print(3)
    try:
        AllChem.MMFFOptimizeMolecule(mol)  # MMFF94
    except:
        AllChem.UFFOptimizeMolecule(mol)  # Fallback на UFF

    # 4. Сохраняем во временный PDB-файл (RDKit не поддерживает PDBQT)
    print(4)
    temp_pdb = "temp_rdkit.pdb"
    Chem.MolToPDBFile(mol, temp_pdb)

    # 5. Конвертируем PDB → PDBQT через Open Babel
    print(5)
    obabel_cmd = f"obabel {temp_pdb} -O {output_pdbqt} -xh --partialcharge gasteiger"
    subprocess.run(obabel_cmd, shell=True, check=True)

    print(f"Файл сохранен: {output_pdbqt}")

# Пример использования
# pdb_to_pdbqt_rdkit("input.pdb", "output.pdbqt")

In [ ]:
pdb_to_pdbqt_rdkit('./docking/294-BD_AD_receptor_ligand_140.988_160.730_-16.038/receptor.pdb', './docking/receptor_auto.pdbqt')

In [ ]:
from preprocess import Zinc_Processor, SmilesPreprocessor

In [ ]:
sp = SmilesPreprocessor(add_Hs=False, kekulize=True, max_atoms=38, max_size=40)
x, A, sz, _ = sp.process('CC(C)(C)c1ccc2occ(CC(=O)Nc3ccccc3F)c2c1')

In [ ]:
np.load('./docking/Марк/zinc_DS/2025-04-12_16-29-13/docking_scores_0-9.npy', allow_pickle=True)

In [ ]:
zp = Zinc_Processor('DMNP_read.txt', './my_scripts')

In [ ]:
np.concat([np.empty(0), np.load(path + file, allow_pickle=True)])

In [ ]:
np.load('./docking/DS_data/25k/small/docking_scores_27630-27639.npy', allow_pickle=True)[:1000] == np.load('./docking/DS_data/25k/small/rdy/docking_scores_27000-27999.npy', allow_pickle=True)[630:640]

In [ ]:
path = './docking/DS_data/25k/small/'
files = sorted(os.listdir(path))
a = np.empty(0, dtype=float)
for i, file in enumerate(files):
    a_new = np.load(path + file, allow_pickle=True)
    a = np.concat([a, a_new])
    if (i + 1) % 100 == 0:
        file_name = f'docking_scores_{25000 + (i - 99) * 10}-{25000 + i * 10 + 9}.npy'
        a.dump(path + 'rdy/' + file_name)
        print('Size of ', file_name, ': ', a.shape[0])
        a = np.empty(0)

In [ ]:
for i in range(5):
    with open('mse.txt', 'a') as f:
        f.write(f'{i},')

In [ ]:
from utils import valid_mol

In [ ]:
import moses
import pandas as pd

In [ ]:
train = moses.get_dataset('train')
test = moses.get_dataset('test')
test_scaffolds = moses.get_dataset('test_scaffolds')

In [ ]:
from torch.utils.data import DataLoader
from moses import CharVocab, StringDataset

train = moses.get_dataset('train')
vocab = CharVocab.from_data(train)
train_dataset = StringDataset(vocab, train)
train_dataloader = DataLoader(
    train_dataset, batch_size=512,
    shuffle=True, collate_fn=train_dataset.default_collate
)

In [ ]:
pd.Series(train)#.to_csv('zinc1500K.smi', index=False)

In [ ]:
for i, (with_bos, with_eos, lengths) in enumerate(train_dataloader):
    print(with_bos.shape, with_eos.shape, len(lengths))
    if i == 2:
        break

In [ ]:
type(train), type(vocab), type(train_dataset), type(train_dataloader)

In [ ]:
train.shape

In [ ]:
from torch.utils.data import DataLoader
from moses import CharVocab, StringDataset

train = moses.get_dataset('train')
vocab = CharVocab.from_data(train)
train_dataset = StringDataset(vocab, train)
train_dataloader = DataLoader(
    train_dataset, batch_size=512,
    shuffle=True, collate_fn=train_dataset.default_collate
)

for with_bos, with_eos, lengths in train_dataloader:
    ...

In [ ]:
from envs import molecular_metrics as mm

In [ ]:
mm.MolecularMetrics._compute_SAS(mol)

In [ ]:
smiles_list[18]

In [ ]:
from rdkit import Chem
from envs.sascorer import calculateScore

smiles1 = "CC1CCC(C2=C1C=CC(=C2)C)C(C)CCC(=O)O"
mol1 = Chem.MolFromSmiles(smiles1)
sa_score1 = calculateScore(mol1)
print(f"SA Score для {smiles1}: {sa_score1}")

smiles2 = "Cc1c(NC(=O)CSc2nc3sc4c(c3c(=O)[nH]2)CCCC4)c(=O)n(-c2ccccc2)n1C"
mol2 = Chem.MolFromSmiles(smiles2)
sa_score2 = calculateScore(mol2)
print(f"SA Score для {smiles2}: {sa_score2}")

In [ ]:
from rdkit import Chem, DataStructs
from rdkit.Chem import AllChem
import rdkit
import rdkit.Chem

fp1 = AllChem.GetMorganFingerprint(mol1, 2)
fp2 = AllChem.GetMorganFingerprint(mol2, 2)

sim = DataStructs.TanimotoSimilarity(fp1, fp2)
print('Танимото схожесть: {}'.format(sim))

In [ ]:
fp = open('./dataset/zinc250k/zinc250k.smi', 'r')
smiles_list = [smiles.strip() for smiles in fp]

In [ ]:
valid_mol(mol)

In [ ]:
mol = Chem.MolFromSmiles('C1=CC=CC=C1')
sml = Chem.MolToSmiles(mol)
print(sml)

In [ ]:
from rdkit import Chem
from rdkit.Chem import Draw

# Пример SMILES
smiles = 'CC(=O)O'  # бензол

# Создание молекулы из SMILES
mol = Chem.MolFromSmiles(smiles)

# Рисование молекулы и сохранение изображения
img = Draw.MolToFile(mol, 'molecule.png')
